In [6]:
import pickle
data = pickle.load(open("../Data/CMU2X_dat.pickle", "rb"))



import tensorflow as tf
from CalculateEER import SKGetEER as GetEER
from sklearn.model_selection import train_test_split
import numpy as np

X = np.array(data['data'])
Y = np.array(data['labels'])
X = X.reshape(X.shape[0], 1, 31, 1)

n = X.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)
print(X_train.shape)

tf.reset_default_graph()
inp = tf.placeholder(tf.float32, shape=[None, 1, 31, 1])
out = tf.placeholder(tf.float32, shape = [None, 51])


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([1, 5, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(inp, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([1, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

print(h_pool2.shape)
W_fc1 = weight_variable([1 * 8 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 1*8*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 51])
b_fc2 = bias_variable([51])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_ = tf.nn.softmax(y_conv)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=out, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(out, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()
try:
    sess.run(tf.global_variables_initializer())
    batchSize = 128
    for epoch in range(500):
        j = -1
        i = 0
        while i < X_train.shape[0]:
            j += 1
            batch = X_train[i: i + batchSize]
            batch_labels = y_train[i: i + batchSize]
            i = i + batchSize
            #print(batch.shape)
            train_step.run(feed_dict={inp: batch, out: batch_labels, keep_prob: 0.5}, session=sess)
            if not j%50:
                train_accuracy = accuracy.eval(feed_dict={inp: batch, out: batch_labels, keep_prob: 1.0}, session=sess)
                train_out = sess.run(y_, feed_dict={inp: batch, out: batch_labels, keep_prob: 1.0})
                valid_accuracy = accuracy.eval(feed_dict={inp: X_valid, out: y_valid, keep_prob: 1.0}, session=sess)
                valid_out = sess.run(y_, feed_dict={inp: X_valid, out: y_valid, keep_prob: 1.0})
                #print(valid_out[0])
                test_accuracy = accuracy.eval(feed_dict={inp: X_test, out: y_test, keep_prob: 1.0}, session=sess)
                test_out = sess.run(y_, feed_dict={inp: X_test, out: y_test, keep_prob: 1.0})
                TEER = GetEER(test_out, y_test)
                VEER = GetEER(valid_out, y_valid)
                print(epoch, j, train_accuracy, valid_accuracy, test_accuracy, VEER, TEER)
except Exception as e:
    print(e)
    d = {}

(42840, 1, 31, 1)
(?, 1, 8, 64)
0 0 0.0078125 0.021023965 0.01906318 0.46475746475746477 0.46243739565943237
0 50 0.1015625 0.09302832 0.08921569 0.3402908848673254 0.33984895601954684
0 100 0.09375 0.090631805 0.08997821 0.26666666666666666 0.2637642627672538
0 150 0.1171875 0.12200436 0.11525054 0.2128982128982129 0.20988203872690853
0 200 0.1640625 0.19084968 0.18409586 0.18722222222222223 0.18536206130608618
0 250 0.2421875 0.22875817 0.23202614 0.16664817485853767 0.1660188784008884
0 300 0.3671875 0.3191721 0.31873637 0.14593993325917687 0.14462120369340306
1 0 0.3515625 0.3261438 0.32930282 0.13518312985571587 0.13448084397556914
1 50 0.375 0.38965142 0.39008716 0.12062645784738421 0.11934156378600823
1 100 0.4609375 0.467756 0.46764705 0.11362879040319894 0.11238200999444753
1 150 0.5078125 0.5114379 0.50958604 0.10753165963119307 0.10747196624847341
1 200 0.5 0.5302832 0.5288671 0.1032952402085876 0.10321432543654765
1 250 0.5390625 0.53878 0.5332244 0.10083194675540766 0.1009

KeyboardInterrupt: 